In [1]:
!pip install numpy==1.26.4 transformers==4.39.3 accelerate==0.21.0 datasets pandas --quiet

In [3]:
import numpy as np
import transformers
import accelerate

print("NumPy:", np.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)

W0622 12:50:26.263000 28596 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


NumPy: 1.26.4
Transformers: 4.39.3
Accelerate: 0.21.0


In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
from datasets import Dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments
)

In [7]:
df = pd.read_csv("ProblemSolutionPythonV3.csv")
df = df.dropna(subset=["Problem", "Python Code"])
df["text"] = df["Problem"] + "\n\n" + df["Python Code"]
df = df[["text"]]
df.head()

,text
0,Write a NumPy program to repeat elements of an...
1,Write a Python function to create and print a ...
2,Write a Python program to remove duplicates fr...
3,Write a NumPy program to compute the x and y c...
4,Write a Python program to alter a given SQLite...


In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

C:\Users\sanke\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

Map:   0%|          | 0/3306 [00:00<?, ? examples/s]

In [16]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

C:\Users\sanke\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding(50257, 768)

In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-epoch-final",       
    logging_dir="./logs",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,                     # Train for 3 epochs
    save_strategy="no",                     # 🚫 No intermediate checkpoints
    evaluation_strategy="epoch",            # You can still evaluate per epoch
    logging_steps=10,
    warmup_steps=10,
    weight_decay=0.01,
    push_to_hub=False,
    no_cuda=True                            # Set to True for CPU
)

C:\Users\sanke\Anaconda3\Lib\site-packages\transformers\training_args.py:1399: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

TypeError: Accelerator.__init__() got an unexpected keyword argument 'use_seedable_sampler'

In [22]:
trainer.train()


NameError: name 'trainer' is not defined

In [26]:
model.save_pretrained("./gpt2-epoch1-checkpoint")
tokenizer.save_pretrained("./gpt2-epoch1-checkpoint")

('./gpt2-epoch1-checkpoint\\tokenizer_config.json',
 './gpt2-epoch1-checkpoint\\special_tokens_map.json',
 './gpt2-epoch1-checkpoint\\vocab.json',
 './gpt2-epoch1-checkpoint\\merges.txt',
 './gpt2-epoch1-checkpoint\\added_tokens.json')

In [18]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("./gpt2-epoch1-checkpoint")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-epoch1-checkpoint")

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the model & tokenizer from checkpoint
model = GPT2LMHeadModel.from_pretrained("./gpt2-epoch1-checkpoint")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-epoch1-checkpoint")

model.eval()

# Set up your test prompt
prompt = "def factorial(s):\n"

# Tokenize and prepare input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate text
with torch.no_grad():
    output = model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
        do_sample=True,       # Set False for deterministic output
        top_k=50,             # Sampling controls
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode and display result
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

def factorial(s):
   factorial(s, 1)
   sum = 0
fact = 0
while (fact == 1):
    sum += 1
    fact += 1
   fact += 2
fact = 1
fact += 2
fact += 3
fact += 4
fact += 5
fact += 6
fact += 7
fact += 8
   fact += 9
   fact += 10
   fact += 11
   fact += 12
   fact += 13
